In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

## Reading original dataset

In [2]:
real= pd.read_csv('../gr_id.csv',sep='\t') 
fake= pd.read_csv('../gf_id.csv', sep='\t')

In [26]:
real.shape, fake.shape

((15931, 15), (4567, 15))

In [3]:
real['label']=1
fake['label']=0

In [4]:
dataset=pd.concat([real,fake])
dataset.shape

(20498, 15)

In [5]:
dataset.columns

Index(['Unnamed: 0', 'url', 'text', 'images', 'top_img', 'keywords', 'authors',
       'canonical_link', 'title', 'meta_data', 'movies', 'publish_date',
       'source', 'summary', 'label'],
      dtype='object')

In [6]:
dataset.shape

(20498, 15)

In [7]:
dataset.head(2)

,Unnamed: 0,url,text,images,top_img,keywords,authors,canonical_link,title,meta_data,movies,publish_date,source,summary,label
0,0,https://www.brides.com/story/teen-mom-jenelle-...,When you get married on national TV (and who i...,['https://media.brides.com/photos/59ca68fc3d68...,https://media.brides.com/photos/59ca68fc3d6836...,[],[],https://www.brides.com/story/teen-mom-jenelle-...,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,"{'msapplication-tap-highlight': 'no', 'viewpor...",[],NaN,https://www.brides.com,NaN,1
1,0,https://www.dailymail.co.uk/tvshowbiz/article-...,Kylie Jenner reportedly doesn't want to talk a...,['https://i.dailymail.co.uk/1s/2019/04/30/22/1...,https://i.dailymail.co.uk/i/pix/2017/07/05/21/...,[],['Ross Mcdonagh For Dailymail.Com'],https://www.dailymail.co.uk/tvshowbiz/article-...,Kylie Jenner refusing to discuss Tyga on Life ...,"{'keywords': 'dailymail,TV&Showbiz,Tyga,Kylie ...",[],1.499290e+09,https://www.dailymail.co.uk,NaN,1


### cleaning dataset

In [24]:
import ast
with open('gossip/demo_real.txt', 'r') as data:
    d = ast.literal_eval(data.read())

In [25]:
poli_top=pd.DataFrame(list(d.items()), columns=['url', 'top_img'])

In [10]:
poli_top.shape

(15071, 2)

In [27]:
with open('gossip/demo_fake.txt', 'r') as data:
    d = ast.literal_eval(data.read())

In [28]:
poli_fake=pd.DataFrame(list(d.items()), columns=['url', 'top_img'])

In [29]:
poli_fake.shape

(4204, 2)

In [104]:
poli_image=pd.concat([poli_top,poli_fake])

In [105]:
tempDataset = dataset.values.tolist()
tempPoli = poli_image.values.tolist()

In [108]:
len(tempDataset), len(tempPoli)

(20498, 19275)

In [109]:
data=(dataset['title']+ ' '+ dataset['text']).tolist()
label=dataset['label'].tolist()
links = dataset['url'].tolist()
labelLinks = [(label[i], links[i]) for i in range(len(label))]

In [110]:
len(labelLinks)

20498

In [114]:
train_data, test_data, train_label,test_label = train_test_split(data, labelLinks,test_size=0.2, random_state=42)

In [115]:
"""train_embeddings = np.array()
test_embeddings = np.load('poli_embed_test.npy')"""

"train_embeddings = np.array()\ntest_embeddings = np.load('poli_embed_test.npy')"

In [116]:
len(train_data) + len(test_data)

20498

In [45]:
"""trainEmbeddingJson = {}
testEmbeddingJson = {}
for i in range(len(train_embeddings)):
    trainEmbeddingJson[train_data[i]] = (train_embeddings[i], train_label[i])
for i in range(len(test_embeddings)):
    testEmbeddingJson[test_data[i]] = (test_embeddings[i], test_label[i])"""

In [117]:
imageDict = {}
for i in tempPoli:
    imageDict[i[0]] = i[1]

In [118]:
len(imageDict)

19275

In [119]:
import shutil

In [120]:
trainCounter = 0
testCounter = 0

In [121]:
train_label[0]

(1,
 'https://www.theguardian.com/music/2018/mar/27/prince-had-exceedingly-high-level-of-fentanyl-in-body-when-he-died')

In [122]:
"""for i in testEmbeddingJson_withImage:
    print(testEmbeddingJson_withImage[i])
    break"""

'for i in testEmbeddingJson_withImage:\n    print(testEmbeddingJson_withImage[i])\n    break'

In [123]:
trainEmbeddingJson_withImage = {}
for i in range(len(train_data)):
    try:
        if imageDict[train_label[i][1]] != {}:
            if os.path.isfile('/home/karmanya/finalExperiment/gossip/merged/' + imageDict[train_label[i][1]]['Top_img'] + '.jpg'):
                trainCounter += 1
                try:
                    shutil.copy('/home/karmanya/finalExperiment/gossip/merged/' + imageDict[train_label[i][1]]['Top_img'] + '.jpg', '/home/karmanya/finalExperiment/imageDataset/gossip/train/' + imageDict[train_label[i][1]]['Top_img'] + '.jpg')
                except Exception as f:
                    print(f)
                trainEmbeddingJson_withImage[train_data[i]] = [train_label[i][0], train_label[i][1], imageDict[train_label[i][1]]]
    except Exception as e:
        #print(e)
        pass

In [124]:
testEmbeddingJson_withImage = {}
for i in range(len(test_data)):
    try:
        if imageDict[test_label[i][1]] != {}:
            if os.path.isfile('/home/karmanya/finalExperiment/gossip/merged/' + imageDict[test_label[i][1]]['Top_img'] + '.jpg'):
                testCounter += 1
                shutil.copy('/home/karmanya/finalExperiment/gossip/merged/' + imageDict[test_label[i][1]]['Top_img'] + '.jpg', '/home/karmanya/finalExperiment/imageDataset/gossip/test/' + imageDict[test_label[i][1]]['Top_img'] + '.jpg')
                testEmbeddingJson_withImage[test_data[i]] = [test_label[i][0], test_label[i][1], imageDict[test_label[i][1]]]
    except Exception as e:
        #print(e)
        pass

In [125]:
trainCounter, testCounter

(15632, 3894)

In [57]:
for i in trainEmbeddingJson_withImage:
    temp = trainEmbeddingJson_withImage[i]
    trainEmbeddingJson_withImage[i] = [temp[1][0], temp[1][1], imageDict[temp[1][1]]]

In [59]:
for i in testEmbeddingJson_withImage:
    temp = testEmbeddingJson_withImage[i]
    testEmbeddingJson_withImage[i] = [temp[1][0], temp[1][1], imageDict[temp[1][1]]]

In [126]:
with open('imageDataset/gossip/trainJson.json', 'w') as f:
    json.dump(trainEmbeddingJson_withImage, f)
with open('imageDataset/gossip/testJson.json', 'w') as f:
    json.dump(testEmbeddingJson_withImage, f)

In [127]:
with open('imageDataset/gossip/imageLinkMapping.json', 'w') as f:
    json.dump(imageDict, f)

In [22]:
"""duplicates = []
for i in tempDataset:
    if i[1] in duplicates:
        print(i[1])
    else:
        duplicates.append(i[1])"""

'duplicates = []\nfor i in tempDataset:\n    if i[1] in duplicates:\n        print(i[1])\n    else:\n        duplicates.append(i[1])'

In [351]:
for i in tempDataset:
    for j in tempPoli:
        if i[1] == j[0]:
            updatedDataset.append(i + [j[1]])

In [353]:
#updatedDataset = pd.DataFrame(updatedDataset)

In [354]:
#updatedDataset.columns = newFields

In [357]:
len(tempPoli), len(updatedDataset)

(661, 679)

In [360]:
len(train_data) + len(test_data)

744

In [361]:
len(train_embeddings) + len(test_embeddings)

744

In [362]:
len(trainEmbeddingJson) + len(testEmbeddingJson)

574

In [278]:
trainEmbeddingJson = {}
testEmbeddingJson = {}
for i in range(len(train_embeddings)):
    trainEmbeddingJson[train_data[i]] = (train_embeddings[i], train_label[i])
for i in range(len(test_embeddings)):
    testEmbeddingJson[test_data[i]] = (test_embeddings[i], test_label[i])

In [281]:
import shutil

In [282]:
trainCounter = 0
testCounter = 0

In [289]:
len(updatedDataset), len(trainEmbeddingJson) + len(testEmbeddingJson)

(679, 574)

In [283]:
for i in updatedDataset:
    if i[-1] != {}:
        temp = i[-8]+ ' '+ i[2]
        if temp in trainEmbeddingJson:
            trainCounter += 1
        elif temp in testEmbeddingJson:
            testCounter += 1
    elif i[-1] == {}:
        temp = i[-8]+ ' '+ i[2]
        if temp in trainEmbeddingJson:
            del trainEmbeddingJson[temp]
        elif temp in testEmbeddingJson:
            del testEmbeddingJson[temp]

In [284]:
trainCounter, testCounter

(452, 110)

In [285]:
len(trainEmbeddingJson), len(testEmbeddingJson)

(455, 119)

In [14]:
sentences = []
with open('training_sentences.txt', 'r') as f:
    lines = f.readlines()

In [15]:
lines[0]

'MITT ROMNEY: ‘TRUMP HAS SOME VERY SERIOUS EMOTIONAL PROBLEMS, HE MUST RESIGN BEFORE WE…’ AMID AN INTERVIEW ON CNN’S “AC360,” FORMER PRESIDENTIAL CANDIDATE MITT ROMNEY URGED PRESIDENT TRUMP TO LEAVE OFFICE OVER THE SEXUAL MISCONDUCT CLAIMS AGAINST HIM AND COMMENTED THAT THE PRESIDENT “HAS SOME VERY SERIOUS EMOTIONAL PROBLEMS.”\n'

In [38]:
URL_poli=dataset['url'].tolist()
TITLE_poli=dataset['title'].tolist()

In [39]:
path='/media/data_dump_2/Shivangi/google_drive_images/Images/politifact/image_ids_to_articles_poli_top_real.txt'

In [59]:
import ast
with open('demo.txt', 'r') as data:
    d= ast.literal_eval(data.read())       

In [60]:
poli_top=pd.DataFrame(list(d.items()), columns=['url', 'top_img'])

In [67]:
poli_top

,url,top_img
0,http://www.nfib-sbet.org/,{'Top_img': 'TNvleUERVcteFhXLFMOhJNNKtVJ7Gvtj'}
1,http://www.cq.com/doc/newsmakertranscripts-494...,{'Top_img': 'F5EvpyGbVWnmGxY51SFwiwFApug6uCQD'}
2,https://web.archive.org/web/20110811143753/htt...,{'Top_img': 'DYJ719DYWZuuhTiYC24sKe8j4xtXAmSS'}
3,https://web.archive.org/web/20070820164107/htt...,{'Top_img': 'nhnAUdrFS91vNVbtAC9GJHYkCcJRJEWl'}
4,http://www.politifact.com/truth-o-meter/statem...,{'Top_img': 'B8gcMTZ9WskrcKCyVXtLr1df9gUbArSB'}
5,https://www.law.cornell.edu/constitution/amend...,{'Top_img': 'wV4ZL7x32sUA3ywBfOj4bqt1FK7modqs'}
6,https://web.archive.org/web/20090913221204/htt...,{'Top_img': 'y7ZQMDtegVuKlqLfU0fDRjzaeVOT9mf8'}
7,https://web.archive.org/web/20080623122709/htt...,{'Top_img': 'GBv7hlj158k63w8HmPEJRuESLwqxgFrv'}
8,http://www.cbsnews.com/blogs/2010/02/23/politi...,{'Top_img': '9HDdEGQ17SAzBEOlvzDnrzd8OdtzczjB'}
9,http://data.bls.gov/timeseries/CES0500000001?o...,{'Top_img': 'SQ3WBpBmtpwTv1xWkaAwHIq53s6mYnUP'}


In [65]:
df = (dataset[['url']].merge(poli_top, on='url', how='left'))

In [66]:
df

,url,top_img
0,http://www.nfib-sbet.org/,{'Top_img': 'TNvleUERVcteFhXLFMOhJNNKtVJ7Gvtj'}
1,http://www.cq.com/doc/newsmakertranscripts-494...,{'Top_img': 'F5EvpyGbVWnmGxY51SFwiwFApug6uCQD'}
2,https://web.archive.org/web/20080204072132/htt...,NaN
3,https://web.archive.org/web/20110811143753/htt...,{'Top_img': 'DYJ719DYWZuuhTiYC24sKe8j4xtXAmSS'}
4,https://web.archive.org/web/20070820164107/htt...,{'Top_img': 'nhnAUdrFS91vNVbtAC9GJHYkCcJRJEWl'}
5,http://www.politifact.com/truth-o-meter/statem...,{'Top_img': 'B8gcMTZ9WskrcKCyVXtLr1df9gUbArSB'}
6,https://www.law.cornell.edu/constitution/amend...,{'Top_img': 'wV4ZL7x32sUA3ywBfOj4bqt1FK7modqs'}
7,https://web.archive.org/web/20090913221204/htt...,{'Top_img': 'y7ZQMDtegVuKlqLfU0fDRjzaeVOT9mf8'}
8,https://web.archive.org/web/20080623122709/htt...,{'Top_img': 'GBv7hlj158k63w8HmPEJRuESLwqxgFrv'}
9,http://www.cbsnews.com/blogs/2010/02/23/politi...,{'Top_img': '9HDdEGQ17SAzBEOlvzDnrzd8OdtzczjB'}
